In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick
# import scipy.optimize as opt
# import scipy.stats as stat
# from mpl_toolkits.mplot3d import Axes3D
# from operator import itemgetter
# import random
import numpy as np
import numpy.ma as ma
# import numpy.linalg as la
# from mpl_toolkits.mplot3d import Axes3D

# pi = np.pi
# sin = np.sin
# cos = np.cos

In [2]:
import numpy as np
from MSD_utils import get_data_gels, build_time_array, return_average, avg_all
from MSD_utils import SD_all, return_SD

In [3]:
path = "./{concentration}_ACSF/geoM2xy_{sample_name}.csv"
frames = 90
SD_frames = [1, 7, 14, 15]
conversion = (0.16, 9.89, 1)
to_frame = 16

data = {}
avg_sets = {}
all_avg = {}
all_SD = {}

data, avg_sets = get_data_gels(["RED", "YG"], ["PEG", "nPEG"], ["in_agarose"], ["0_1x", "1x", "10x"], ["S1", "S2", "S3", "S4"], path);
time, time_SD = build_time_array(frames, conversion, SD_frames)
all_avg = avg_all(data, frames, avg_sets)
all_SD = SD_all(data, frames, SD_frames, avg_sets)

In [4]:
# def range_and_ticks(range_to_graph, to_frame, manual_decimals = False, manual_decimals_val = 2):
#     """
#     This is a useful function that calculates the best range, tick mark interval size, and decimals displayed for a given input
#     dataset.
    
#     Inputs:
#     range_to_graph: numpy array of data to be graphed.
#     to_frame: integer, limits data to be graphed to the range [0, to_frame].
#     manual_decimals: True/False.  Allow the user to manually adjust the number of decimals displayed.
#     manual_decimals_val: integer.  Number of decimals the user desires.
    
#     Outputs:
#     y_range: upper boundary containing the entire range of range_to_graph[0, to_frame].
#     ticks: tick interval.
#     decimals: number of decimals to display.
    
    
    
#     This function must be modified if I need to plot graphs with negative values.
#     I should also include symmetrical capabilities when I want to plot negative values.
#     """
#     graph_max = np.nanmax(range_to_graph[0:to_frame]) #Define max within range

#     if np.ceil(np.log10(graph_max)) >= 0: #Find nearest 10^x that contains graph_max
#         base = np.ceil(np.log10(graph_max))
#         raw_max = 10**base
#         decimals = int(1)
#     else:
#         base = np.ceil(np.log10(1/graph_max))
#         raw_max = 10**(-base)
#         decimals = int(base + 1)

#     range_correct = False
#     cut_down_max = 0.1*raw_max
#     counter = -1
#     while range_correct is False: #Make sure that most of the graph space is used efficiently (75%).
#         if graph_max/raw_max > 0.75:
#             y_range = raw_max
#             range_correct = True
#         else:
#             raw_max = raw_max - cut_down_max
#             range_correct = False
#         counter = counter + 1

#     if graph_max > y_range: #Make sure that graph_max doesn't exceed limits
#         y_range = y_range + cut_down_max

#     if counter % 2 == 0: #Define tick size. I based it off of 0.1*raw_max, which is always a 10^x
#         ticks = cut_down_max * 2
#     else:
#         ticks = cut_down_max

#     range_correct = False #Modifies the tick size if there aren't enough ticks on the graph.
#     while range_correct is False:
#         if ticks/y_range >= 0.24:
#             ticks = ticks/2
#             range_correct = False
#         else:
#             range_correct = True
    
#     if manual_decimals == True: #Allow user to set manual decimal values.
#         decimals = manual_decimals_val
    
#     return y_range, ticks, decimals

In [5]:
y_range, ticks_y, dec_y = range_and_ticks(all_avg[avg_sets[3]]*1, 15)

print('y range is ', y_range)
print('tick size is', ticks_y)
print('decimals', dec_y)


y range is  4.0
tick size is 0.5
decimals 1


In [6]:
x_range, ticks_x, dec_x = range_and_ticks(time, 15, manual_decimals = True, manual_decimals_val = 2)

print('x range is ', x_range)
print('tick size is', ticks_x)
print('decimals', dec_x)

x range is  2.0
tick size is 0.25
decimals 2


In [7]:
def choose_y_axis_params(all_avg, in_name1, in_name2, to_frame):
    """
    When plotting multiple trajectories, I need to choose plot parameters based on multiple datasets.  This function uses the
    function ranges_and_ticks to choose the best parameters based on multiple trajectories.
    
    Inputs:
    all_avg: dictionary of numpy arrays.  A subset of this data will be plotted based on the parameters in_name1 and in_name2.
    in_name1: string.  in_name1 should be found within the keys in all_avg of all datasets to be plotted e.g. if I want to plot 
        all RED datasets, the I could use in_name1=RED.
    in_name2: similar criteria to in_name1.  If I want to narrow the data to be plotted to PEG datasets, then, in_name2 could be
        "_PEG" (if you only use PEG, it will plot all nPEG and PEG datasets, due to my chosen nomenclature. Just be aware of
        your naming system).
    to_frame: integer, limits data to be graphed to the range [0, to_frame].
    
    Outputs:
    y_range_final: upper boundary containing the entire range of range_to_graph[0, to_frame].
    ticks_final: tick interval.
    decimals_final: number of decimals to display.
    
    Example:
    choose_y_axis_params(all_avg, "RED", "_PEG", 15)
    """

    to_graph = {}
    counter = 0

    for keys in all_avg:
        if in_name1 in keys and in_name2 in keys:
            to_graph[counter] = keys
            counter = counter + 1

    y_range = np.zeros(counter)
    ticks = np.zeros(counter)
    decimals = np.zeros(counter)
    base = np.zeros(counter)

    for keys in to_graph:
        y_range[keys], ticks[keys], decimals[keys] = range_and_ticks(all_avg[to_graph[keys]], to_frame)

    one_to_graph = np.argmax(y_range)
    y_range_final = y_range[one_to_graph]
    ticks_final = ticks[one_to_graph]
    decimals_final = int(decimals[one_to_graph])

    return y_range_final, ticks_final, decimals_final

In [10]:
def graph_single_variable(all_avg, time, time_SD, SD_frames, in_name1, in_name2, to_frame=15, line_colors=['g', 'r', 'b', 'c', 'm', 'k'], line_kind='-',
                         x_manual_decimals=False, x_manual_decimals_val=2, label_size=95, legend_size=40, tick_size=50,
                         line_width=10, fig_size=(20,18), modify_labels=False, label_identifier="agarose_"):
    to_graph = {}
    line_type = {}
    counter = 0
    labels = {}

    y_range, ticks_y, dec_y = choose_y_axis_params(all_avg, in_name1, in_name2, to_frame)
    x_range, ticks_x, dec_x = range_and_ticks(time, to_frame, x_manual_decimals, x_manual_decimals_val)

    filename = in_name1 + in_name2 + ".png"

    # Creates figure
    fig = plt.figure(figsize=fig_size, dpi=80)
    ax = fig.add_subplot(111)

    for keys in all_avg:
        if in_name1 in keys and in_name2 in keys:
            to_graph[counter] = keys
            counter = counter + 1

    for keys in to_graph:
        if modify_labels==True:
            labels[keys] = to_graph[keys].split(label_identifier, 1)[1]
        else:
            labels[keys] = to_graph[keys]
        line_type[keys] = line_colors[keys]+line_kind  
        ax.plot(time[0:to_frame], all_avg[to_graph[keys]][0:to_frame], line_type[keys], linewidth=line_width, label=labels[keys])
        ax.errorbar(time_SD, all_avg[to_graph[keys]][SD_frames], all_SD[to_graph[keys]], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color=line_colors[keys])

    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(tick_size)

    xmajor_ticks = np.arange(0, x_range+0.0001, ticks_x)
    ymajor_ticks = np.arange(0, y_range+0.0001, ticks_y)

    ax.set_xticks(xmajor_ticks)
    plt.xticks(rotation=-30)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(tick_size)
    ax.set_xlabel('Time (s)', fontsize=label_size)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=label_size)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.02, 0.75), prop={'size': legend_size})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec_x)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec_y)))


    # plt.yscale('log')
    # plt.xscale('log')
    plt.gca().set_xlim([0, x_range+0.0001])
    plt.gca().set_ylim([0, y_range+0.0001])

    # Save your figure
    plt.savefig('{}'.format(filename), bbox_inches='tight')

In [11]:
in_name1 = "RED"
in_name2 = "nPEG"

graph_single_variable(all_avg, time, time_SD, SD_frames, in_name1, in_name2, to_frame=to_frame,  x_manual_decimals=True, x_manual_decimals_val=2,
                      modify_labels=True, label_identifier="agarose_")
plt.show()

In [12]:
in_name1 = "RED"
in_name2 = "_PEG"

graph_single_variable(all_avg, time, time_SD, SD_frames, in_name1, in_name2, to_frame=to_frame,  x_manual_decimals=True, x_manual_decimals_val=2,
                      modify_labels=True, label_identifier="agarose_")
plt.show()